## Step 1: Setup Google Colab Environment

Configure GPU acceleration and verify CUDA availability.

In [ ]:
# ============================================================================
# SECTION 1: Setup Google Colab Environment
# ============================================================================
import torch
import os

print(f"GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU Device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA Version: {torch.version.cuda}")
else:
    print("⚠️  No GPU detected. Training will be slow on CPU.")

import random
import numpy as np

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

print("\n✓ Environment setup complete!")

## Step 2: Clone Repository and Install Dependencies

Clone PFLlib from GitHub and install required Python packages.

In [ ]:
# ============================================================================
# SECTION 2: Clone Repository and Install Dependencies
# ============================================================================
import subprocess
import sys

print("Cloning PFLlib repository...")
subprocess.run(["git", "clone", "https://github.com/mamintoosi-papers-codes/PFLlib.git",
                "/content/PFLlib"], capture_output=False)

os.chdir("/content/PFLlib")
print(f"Current directory: {os.getcwd()}")

print("\nInstalling required packages...")
subprocess.run([sys.executable, "-m", "pip", "install", "-q", "ujson", "scikit-learn",
                "matplotlib", "seaborn", "h5py", "pandas"], check=False)

print("\n✓ Dependencies installed!")
print(f"\nPFLlib location: {os.getcwd()}")

## Step 3: Generate and Prepare CIFAR-10 Dataset

Download CIFAR-10 and distribute data across multiple clients.

**Configuration:**
- Number of clients: 10
- Data distribution: IID (balanced)
- Dataset: CIFAR-10 (10 classes, 32x32 RGB images)

In [ ]:
# ============================================================================
# SECTION 3: Load and Prepare CIFAR-10 Dataset
# ============================================================================
import sys
sys.path.insert(0, "/content/PFLlib/dataset")

print("Generating CIFAR-10 dataset...")
print("This may take a few minutes on first run (downloading dataset)...\n")

from generate_Cifar10 import generate_dataset

num_clients = 10
dir_path = "/content/PFLlib/dataset/Cifar10/"
niid = False
balance = True
partition = None

generate_dataset(dir_path, num_clients, niid, balance, partition)

print(f"\n✓ Dataset generated successfully!")
print(f"  Number of clients: {num_clients}")
print(f"  Data split: IID={'Yes' if not niid else 'No (Non-IID)'}")
print(f"  Balanced distribution: {balance}")
print(f"  Dataset path: {dir_path}")

## Step 4: Configure Common Parameters

Set up shared parameters for both algorithms.

**Optimized Parameters for Meaningful Comparison:**

- **Communication Rounds: 50** - Allows sufficient time to observe convergence patterns and differences between algorithms
- **Local Epochs: 5** - Standard FL setting balancing local computation and communication
- **Batch Size: 64** - Larger batch size for more stable gradient estimates
- **Learning Rate: 0.01** - Well-tested rate for CIFAR-10 with CNN
- **Join Ratio: 0.5** - Only 50% of clients participate each round, testing robustness under partial participation (key advantage of SR-FedAvg)
- **SR Beta: 0.9** - Momentum coefficient for Stein-Rule shrinkage

**Why These Parameters?**
- Higher rounds reveal convergence speed differences
- Partial participation (50%) creates variance that SR-FedAvg is designed to handle
- Larger batch size reduces noise for cleaner comparison

In [ ]:
# ============================================================================
# SECTION 4: Configure Common Federated Learning Parameters
# ============================================================================
import argparse
import json

# Common configuration for both algorithms
# These parameters are optimized for meaningful comparison
COMMON_CONFIG = {
    'dataset': 'Cifar10',
    'dataset_path': '/content/PFLlib/dataset/Cifar10/',
    'model_type': 'cnn',
    'num_rounds': 50,          # Increased from 20 to see convergence patterns
    'local_epochs': 5,         # Standard setting for FL
    'batch_size': 64,          # Increased for more stable gradients
    'learning_rate': 0.01,     # Standard learning rate for CIFAR-10
    'weight_decay': 1e-4,
    'join_ratio': 0.5,         # 50% client participation - tests robustness
    'eval_gap': 1,             # Evaluate every round for detailed analysis
    'sr_beta': 0.9,            # SR-FedAvg momentum coefficient
}

# Load dataset configuration
config_path = COMMON_CONFIG['dataset_path'] + "config.json"
with open(config_path, 'r') as f:
    dataset_config = json.load(f)

num_clients = dataset_config['num_clients']
num_classes = dataset_config['num_classes']
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("\n" + "="*60)
print("COMPARISON EXPERIMENT CONFIGURATION")
print("="*60)
print(f"Dataset: {COMMON_CONFIG['dataset']}")
print(f"Number of Clients: {num_clients}")
print(f"Number of Classes: {num_classes}")
print(f"\nTraining Settings:")
print(f"  Communication Rounds: {COMMON_CONFIG['num_rounds']}")
print(f"  Local Epochs per Round: {COMMON_CONFIG['local_epochs']}")
print(f"  Batch Size: {COMMON_CONFIG['batch_size']}")
print(f"  Learning Rate: {COMMON_CONFIG['learning_rate']}")
print(f"  Client Participation: {COMMON_CONFIG['join_ratio']*100}%")
print(f"  SR-FedAvg Beta: {COMMON_CONFIG['sr_beta']}")
print(f"  Device: {device}")
print("="*60)
print("\n✓ Configuration complete!")

## Step 5: Helper Function to Create Args

Create a helper function to generate args for each algorithm.

In [ ]:
# ============================================================================
# SECTION 5: Helper Function to Create Args
# ============================================================================
import sys
import os

# Change to system directory and add to path
os.chdir("/content/PFLlib/system")
sys.path.insert(0, "/content/PFLlib/system")

from flcore.trainmodel.models import FedAvgCNN

def create_args(algorithm_name, goal_suffix):
    """Create arguments namespace for an algorithm."""
    args = argparse.Namespace()

    # Dataset config
    args.dataset = COMMON_CONFIG['dataset']
    args.dataset_path = COMMON_CONFIG['dataset_path']
    args.algorithm = algorithm_name
    args.goal = goal_suffix

    # Model
    args.model = FedAvgCNN(in_features=3, num_classes=num_classes, dim=1600).to(device)

    # Training parameters
    args.num_clients = num_clients
    args.num_classes = num_classes
    args.global_rounds = COMMON_CONFIG['num_rounds']
    args.local_epochs = COMMON_CONFIG['local_epochs']
    args.batch_size = COMMON_CONFIG['batch_size']
    args.local_learning_rate = COMMON_CONFIG['learning_rate']
    args.join_ratio = COMMON_CONFIG['join_ratio']
    args.random_join_ratio = False
    args.device = device

    # SR-FedAvg specific
    if algorithm_name == 'SR-FedAvg':
        args.sr_beta = COMMON_CONFIG['sr_beta']

    # Additional required parameters
    args.few_shot = 0
    args.time_select = False
    args.time_threthold = 10000
    args.save_folder_name = "items"
    args.top_cnt = 100
    args.auto_break = False
    args.eval_gap = COMMON_CONFIG['eval_gap']
    args.client_drop_rate = 0.0
    args.train_slow_rate = 0.0
    args.send_slow_rate = 0.0
    args.dlg_eval = False
    args.dlg_gap = 100
    args.batch_num_per_client = 2
    args.num_new_clients = 0
    args.fine_tuning_epoch_new = 0
    args.learning_rate_decay = False
    args.learning_rate_decay_gamma = 0.99

    return args

print("✓ Helper function created!")

## Step 6: Run FedAvg Experiment

Train using standard Federated Averaging algorithm.

In [ ]:
# ============================================================================
# SECTION 6: Run FedAvg Experiment
# ============================================================================
import sys
import time
os.chdir("/content/PFLlib/system")
sys.path.insert(0, "/content/PFLlib/system")

from flcore.servers.serveravg import FedAvg

print("\n" + "="*60)
print("EXPERIMENT 1: FEDAVG")
print("="*60)

# Create args for FedAvg
args_fedavg = create_args('FedAvg', 'comparison')

print("\nInitializing FedAvg server...")
server_fedavg = FedAvg(args_fedavg, times=0)
print(f"✓ Server initialized with {len(server_fedavg.clients)} clients")

print("\nStarting FedAvg training...")
start_time = time.time()
server_fedavg.train()
fedavg_time = time.time() - start_time

print("\n" + "="*60)
print(f"FedAvg Training Completed in {fedavg_time:.2f} seconds")
print("="*60)

# Extract metrics
fedavg_acc = server_fedavg.rs_test_acc
fedavg_loss = server_fedavg.rs_train_loss

print(f"\nFedAvg Results:")
print(f"  Final Test Accuracy: {fedavg_acc[-1]:.4f}")
print(f"  Best Test Accuracy: {max(fedavg_acc):.4f}")
print(f"  Final Train Loss: {fedavg_loss[-1]:.4f}")

## Step 7: Run SR-FedAvg Experiment

Train using SR-FedAvg (FedAvg with Stein-Rule shrinkage).

In [ ]:
# ============================================================================
# SECTION 7: Run SR-FedAvg Experiment
# ============================================================================
from flcore.servers.serversrfedavg import SR_FedAvg

print("\n" + "="*60)
print("EXPERIMENT 2: SR-FEDAVG")
print("="*60)

# Create args for SR-FedAvg
args_srfedavg = create_args('SR-FedAvg', 'comparison')

print("\nInitializing SR-FedAvg server...")
server_srfedavg = SR_FedAvg(args_srfedavg, times=0)
print(f"✓ Server initialized with {len(server_srfedavg.clients)} clients")
print(f"  SR Beta (momentum): {args_srfedavg.sr_beta}")

print("\nStarting SR-FedAvg training...")
start_time = time.time()
server_srfedavg.train()
srfedavg_time = time.time() - start_time

print("\n" + "="*60)
print(f"SR-FedAvg Training Completed in {srfedavg_time:.2f} seconds")
print("="*60)

# Extract metrics
srfedavg_acc = server_srfedavg.rs_test_acc
srfedavg_loss = server_srfedavg.rs_train_loss

print(f"\nSR-FedAvg Results:")
print(f"  Final Test Accuracy: {srfedavg_acc[-1]:.4f}")
print(f"  Best Test Accuracy: {max(srfedavg_acc):.4f}")
print(f"  Final Train Loss: {srfedavg_loss[-1]:.4f}")

## Step 8: Compare Results

Display side-by-side comparison of both algorithms.

In [ ]:
# ============================================================================
# SECTION 8: Compare Results
# ============================================================================
import pandas as pd

print("\n" + "="*60)
print("COMPARISON RESULTS")
print("="*60)

comparison_data = {
    'Metric': [
        'Final Test Accuracy',
        'Best Test Accuracy',
        'Convergence Round',
        'Final Train Loss',
        'Training Time (seconds)'
    ],
    'FedAvg': [
        f"{fedavg_acc[-1]:.4f}",
        f"{max(fedavg_acc):.4f}",
        f"{np.argmax(fedavg_acc) + 1}",
        f"{fedavg_loss[-1]:.4f}",
        f"{fedavg_time:.2f}"
    ],
    'SR-FedAvg': [
        f"{srfedavg_acc[-1]:.4f}",
        f"{max(srfedavg_acc):.4f}",
        f"{np.argmax(srfedavg_acc) + 1}",
        f"{srfedavg_loss[-1]:.4f}",
        f"{srfedavg_time:.2f}"
    ]
}

df_comparison = pd.DataFrame(comparison_data)
print("\n")
print(df_comparison.to_string(index=False))
print("\n" + "="*60)

# Calculate improvement
improvement = ((max(srfedavg_acc) - max(fedavg_acc)) / max(fedavg_acc)) * 100

print(f"\n📊 Performance Improvement:")
print(f"   SR-FedAvg vs FedAvg: {improvement:+.2f}%")

if improvement > 0:
    print(f"   ✓ SR-FedAvg achieved better accuracy!")
elif improvement < -1:
    print(f"   Note: FedAvg performed better in this run")
else:
    print(f"   Both algorithms performed similarly")

print("\n" + "="*60)

## Step 9: Visualize Training Curves

Create detailed comparison plots showing accuracy and loss progression.

In [ ]:
# ============================================================================
# SECTION 9: Visualize Training Curves
# ============================================================================
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (16, 6)

fig, axes = plt.subplots(1, 2, figsize=(16, 6))

rounds = range(1, len(fedavg_acc) + 1)

# Plot 1: Test Accuracy Comparison
ax1 = axes[0]
ax1.plot(rounds, fedavg_acc, 'b-', linewidth=2.5, marker='o', markersize=6,
         label='FedAvg', markevery=2)
ax1.plot(rounds, srfedavg_acc, 'r-', linewidth=2.5, marker='s', markersize=6,
         label='SR-FedAvg', markevery=2)
ax1.set_xlabel('Communication Round', fontsize=13, fontweight='bold')
ax1.set_ylabel('Test Accuracy', fontsize=13, fontweight='bold')
ax1.set_title('Test Accuracy Comparison: FedAvg vs SR-FedAvg',
              fontsize=14, fontweight='bold')
ax1.legend(fontsize=12, loc='lower right')
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1])

# Annotate final values
ax1.annotate(f'FedAvg: {fedavg_acc[-1]:.4f}',
             xy=(len(fedavg_acc), fedavg_acc[-1]),
             xytext=(-80, -25), textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.5', fc='lightblue', alpha=0.8),
             arrowprops=dict(arrowstyle='->', lw=1.5),
             fontsize=10)

ax1.annotate(f'SR-FedAvg: {srfedavg_acc[-1]:.4f}',
             xy=(len(srfedavg_acc), srfedavg_acc[-1]),
             xytext=(-80, 25), textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.5', fc='lightcoral', alpha=0.8),
             arrowprops=dict(arrowstyle='->', lw=1.5),
             fontsize=10)

# Plot 2: Training Loss Comparison
ax2 = axes[1]
ax2.plot(rounds, fedavg_loss, 'b-', linewidth=2.5, marker='o', markersize=6,
         label='FedAvg', markevery=2)
ax2.plot(rounds, srfedavg_loss, 'r-', linewidth=2.5, marker='s', markersize=6,
         label='SR-FedAvg', markevery=2)
ax2.set_xlabel('Communication Round', fontsize=13, fontweight='bold')
ax2.set_ylabel('Training Loss', fontsize=13, fontweight='bold')
ax2.set_title('Training Loss Comparison: FedAvg vs SR-FedAvg',
              fontsize=14, fontweight='bold')
ax2.legend(fontsize=12, loc='upper right')
ax2.grid(True, alpha=0.3)

# Annotate final values
ax2.annotate(f'FedAvg: {fedavg_loss[-1]:.4f}',
             xy=(len(fedavg_loss), fedavg_loss[-1]),
             xytext=(-80, 25), textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.5', fc='lightblue', alpha=0.8),
             arrowprops=dict(arrowstyle='->', lw=1.5),
             fontsize=10)

ax2.annotate(f'SR-FedAvg: {srfedavg_loss[-1]:.4f}',
             xy=(len(srfedavg_loss), srfedavg_loss[-1]),
             xytext=(-80, -25), textcoords='offset points',
             bbox=dict(boxstyle='round,pad=0.5', fc='lightcoral', alpha=0.8),
             arrowprops=dict(arrowstyle='->', lw=1.5),
             fontsize=10)

plt.tight_layout()
plt.savefig('/content/PFLlib/comparison_results.png', dpi=200, bbox_inches='tight')
print("\n✓ Comparison plot saved to: /content/PFLlib/comparison_results.png")
plt.show()

## Step 10: Statistical Analysis

Analyze variance and stability of both algorithms.

In [ ]:
# ============================================================================
# SECTION 10: Statistical Analysis
# ============================================================================

print("\n" + "="*60)
print("STATISTICAL ANALYSIS")
print("="*60)

# Calculate variance in accuracy improvements
fedavg_improvements = np.diff(fedavg_acc)
srfedavg_improvements = np.diff(srfedavg_acc)

fedavg_var = np.var(fedavg_improvements)
srfedavg_var = np.var(srfedavg_improvements)

print(f"\nVariance in Round-to-Round Accuracy Changes:")
print(f"  FedAvg: {fedavg_var:.6f}")
print(f"  SR-FedAvg: {srfedavg_var:.6f}")
print(f"  Variance Reduction: {(1 - srfedavg_var/fedavg_var)*100:.2f}%")

if srfedavg_var < fedavg_var:
    print(f"  ✓ SR-FedAvg shows more stable convergence")
else:
    print(f"  Note: FedAvg showed more stable convergence in this run")

# Final comparison
print(f"\nAccuracy Statistics:")
print(f"  FedAvg - Mean: {np.mean(fedavg_acc):.4f}, Std: {np.std(fedavg_acc):.4f}")
print(f"  SR-FedAvg - Mean: {np.mean(srfedavg_acc):.4f}, Std: {np.std(srfedavg_acc):.4f}")

print("\n" + "="*60)

## Step 11: Summary and Key Findings

Display final summary and insights from the comparison.

In [ ]:
# ============================================================================
# SECTION 11: Summary and Key Findings
# ============================================================================

print("\n" + "="*60)
print("COMPARISON EXPERIMENT COMPLETED SUCCESSFULLY")
print("="*60)

print(f"\n📊 Experimental Setup:")
print(f"   Dataset: CIFAR-10 (10 classes)")
print(f"   Number of Clients: {num_clients}")
print(f"   Client Participation: {COMMON_CONFIG['join_ratio']*100}% per round")
print(f"   Communication Rounds: {COMMON_CONFIG['num_rounds']}")
print(f"   Local Epochs: {COMMON_CONFIG['local_epochs']}")

print(f"\n🏆 Performance Comparison:")
print(f"   FedAvg Best Accuracy: {max(fedavg_acc):.4f}")
print(f"   SR-FedAvg Best Accuracy: {max(srfedavg_acc):.4f}")
print(f"   Improvement: {improvement:+.2f}%")

print(f"\n🔬 Key Findings:")
print(f"   1. SR-FedAvg uses Stein-Rule shrinkage for variance reduction")
print(f"   2. Momentum coefficient (beta={COMMON_CONFIG['sr_beta']}) smooths updates")
print(f"   3. Per-layer shrinkage adapts to parameter importance")

print(f"\n💡 When to Use SR-FedAvg:")
print(f"   ✓ Partial client participation (low join ratio)")
print(f"   ✓ High client heterogeneity")
print(f"   ✓ Need for stable convergence")
print(f"   ✓ Limited communication rounds")

print(f"\n📁 Output Files:")
print(f"   Comparison plot: /content/PFLlib/comparison_results.png")
print(f"   FedAvg results: /content/PFLlib/results/Cifar10_FedAvg_comparison_0.h5")
print(f"   SR-FedAvg results: /content/PFLlib/results/Cifar10_SR-FedAvg_comparison_0.h5")

print("\n" + "="*60)
print("Thank you for using PFLlib!")
print("="*60)

## Optional: Export Results to CSV

Save detailed results for further analysis.

In [ ]:
# ============================================================================
# OPTIONAL: Export Results to CSV
# ============================================================================

# Create detailed results dataframe
detailed_results = pd.DataFrame({
    'Round': list(rounds),
    'FedAvg_Accuracy': fedavg_acc,
    'SR-FedAvg_Accuracy': srfedavg_acc,
    'FedAvg_Loss': fedavg_loss,
    'SR-FedAvg_Loss': srfedavg_loss,
    'Accuracy_Difference': np.array(srfedavg_acc) - np.array(fedavg_acc)
})

# Save to CSV
csv_path = '/content/PFLlib/detailed_comparison_results.csv'
detailed_results.to_csv(csv_path, index=False, float_format='%.4f')

print(f"✓ Detailed results exported to: {csv_path}")
print("\nPreview of results:")
print(detailed_results.head(10))

# Display download link for Colab
from google.colab import files
print("\nTo download the results, run:")
print(f"files.download('{csv_path}')")

In [ ]:
files.download('/content/PFLlib/detailed_comparison_results.csv')